<hr style="height:.9px;border:none;color:#333;background-color:#333;" />
<hr style="height:.9px;border:none;color:#333;background-color:#333;" />
Individual Case

Alban de Raemy

Regression Analysis

MSBA 2

Machine Learning
<hr style="height:.9px;border:none;color:#333;background-color:#333;" />
<hr style="height:.9px;border:none;color:#333;background-color:#333;" />

<h2>Purpose of this Script</h2><br>

This script's purpose is to build a predictive model on the continuous variable (Revenue). The goal is to develop reliable models, compare them and select the best model to predict revenue from a customer in their first year of using Apprentice Chef.
My personal goal is to develop my skills in creating predictive models in python.

<h2>Analytical Objectives</h2><br>

Part 1:
Understand how much revenue Apprentice Chef should expect from each customer within their first year of using the services.

Part 2: Predict which customer will subscribe to the Halfway there service.


<h2>Fundamental Data Exploration</h2><br>

In [1]:
# Import libraries
import pandas as pd                                  # data science essentials
import matplotlib.pyplot as plt                      # essential graphical output
import seaborn as sns                                # enhanced graphical output
from   scipy import stats                            # stats essentials
import statsmodels.formula.api as smf                # predictive modeling with nice outputs
import gender_guesser.detector as gender             # guess gender based on (given) name
import random as rand                                # random
import statsmodels.formula.api as smf                # regression modeling
from sklearn.model_selection import train_test_split # train/test split
from sklearn.linear_model import LinearRegression    # Linear Regression  
from sklearn.neighbors import KNeighborsRegressor    # KNN for Regression
from sklearn.preprocessing import StandardScaler     # standard scaler
import sklearn.linear_model                          # Linear Regression
from itertools import combinations                   # combinations of features


# setting pandas print options
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

# setting random seed
rand.seed(a = 219)

# specifying file name
file = './Apprentice_Chef_Dataset.xlsx'


# reading the file into Python
chef = pd.read_excel(io=file)


# using .shape to view (ROWS, COLUMNS)
chef.shape

# formatting and printing the dimensions of the dataset
#print(f"""
#Size of Original Dataset
#------------------------
#Observations: {chef.shape[0]}
#Features:     {chef.shape[1]}
#""")

# Un-comment the follwing lines for in-deepth analysis
#print(chef.columns)
#chef.head(n=5)
#chef.info() 
# As we can see there is no null variables

# rename LARGEST_ORDER_SIZE to AVG_MEALS_ORDERED
chef.rename(columns={'LARGEST_ORDER_SIZE': 'AVG_MEALS_ORDERED'}, inplace=True)



# Our y-variable is REVENUE
#sns.displot(data = chef,
#           x = "REVENUE",
#           height = 5,
#           aspect = 2)
# displaying the histogram
#plt.show()


We start our analysis with 1946 Observations and 28 Features

<h3>Missing Value Analysis and Imputation</h3><br>

In [2]:
#checking if there is any missing data
#chef.isnull().sum()

There are only missing values in the family name variable (47 of them). We don't need to
imput those missing values because they are irrelevant to our analysis.

<h3>Classify Data</h3><br>

In [2]:
# descriptive statistics for numeric data
chef.describe(include='number').round(2)

,REVENUE,CROSS_SELL_SUCCESS,TOTAL_MEALS_ORDERED,UNIQUE_MEALS_PURCH,CONTACTS_W_CUSTOMER_SERVICE,PRODUCT_CATEGORIES_VIEWED,AVG_TIME_PER_SITE_VISIT,MOBILE_NUMBER,CANCELLATIONS_BEFORE_NOON,CANCELLATIONS_AFTER_NOON,TASTES_AND_PREFERENCES,PC_LOGINS,MOBILE_LOGINS,WEEKLY_PLAN,EARLY_DELIVERIES,LATE_DELIVERIES,PACKAGE_LOCKER,REFRIGERATED_LOCKER,AVG_PREP_VID_TIME,AVG_MEALS_ORDERED,MASTER_CLASSES_ATTENDED,MEDIAN_MEAL_RATING,AVG_CLICKS_PER_VISIT,TOTAL_PHOTOS_VIEWED
count,1946.00,1946.00,1946.00,1946.0,1946.00,1946.00,1946.00,1946.00,1946.00,1946.00,1946.00,1946.00,1946.00,1946.00,1946.00,1946.00,1946.00,1946.00,1946.00,1946.00,1946.00,1946.00,1946.00,1946.00
mean,2107.29,0.68,74.63,4.9,6.98,5.38,99.60,0.88,1.40,0.17,0.71,5.52,1.48,11.33,1.49,2.97,0.36,0.11,150.56,4.44,0.60,2.79,13.51,106.43
std,1138.29,0.47,55.31,2.5,2.28,3.04,62.34,0.33,1.55,0.43,0.45,0.58,0.53,13.57,2.32,2.74,0.48,0.32,49.45,1.55,0.64,0.76,2.33,181.01
min,131.00,0.00,11.00,1.0,1.00,1.00,10.33,0.00,0.00,0.00,0.00,4.00,0.00,0.00,0.00,0.00,0.00,0.00,33.40,1.00,0.00,1.00,5.00,0.00
25%,1350.00,0.00,39.00,3.0,5.00,3.00,72.00,1.00,0.00,0.00,0.00,5.00,1.00,1.00,0.00,1.00,0.00,0.00,114.40,3.00,0.00,2.00,12.00,0.00
50%,1740.00,1.00,60.00,5.0,7.00,5.00,94.16,1.00,1.00,0.00,1.00,6.00,1.00,7.00,0.00,2.00,0.00,0.00,145.60,4.00,1.00,3.00,13.00,0.00
75%,2670.00,1.00,95.00,7.0,8.00,8.00,117.29,1.00,2.00,0.00,1.00,6.00,2.00,13.00,3.00,4.00,1.00,0.00,173.78,5.00,1.00,3.00,15.00,174.00
max,8793.75,1.00,493.00,19.0,18.00,10.00,1645.60,1.00,13.00,3.00,1.00,7.00,3.00,52.00,9.00,19.00,1.00,1.00,564.20,11.00,3.00,5.00,19.00,1600.00


<h3>Visual Data Exploration</h3><br>

We created histograms and boxplots for each features to explore their distribution visually.

In [3]:
# modify text
# subsetting data so that the there's into categorical/descriptive data
chef_num = chef.drop(["NAME", "EMAIL", "FIRST_NAME", "FAMILY_NAME"], axis = 1)
chef_cat = chef.select_dtypes(include = 'object')


# Look at histogram of the numerical variables to better understand their distribution
#for i in chef_num.columns:
#    def plot(x, y, z):
#        fig, ax = plt.subplots(figsize = (12,12))
#        plt.subplot(2,2,y)
#        sns.distplot(chef[x],
#            color = z)
#        plt.xlabel(x)
#        plt.title(x)
#        plt.show()
#        return
#    plot(i, 1, "b")


# Look at boxplot of numerical variables to better understand their distribution
# sns.boxplot(x=chef['AVG_PREP_VID_TIME'])

# creating a list of continuous features (including REVENUE)
#continuous_data = ['REVENUE','AVG_TIME_PER_SITE_VISIT', 'WEEKLY_PLAN', 'AVG_PREP_VID_TIME',
#                   'AVG_MEALS_ORDERED', 'AVG_CLICKS_PER_VISIT', 'TOTAL_PHOTOS_VIEWED']


# developing a correlation matrix based on continuous features
#chef_corr = chef[continuous_data].corr(method = 'pearson')


# filtering the results to only show correlations with Sale_Price
#chef_corr.loc[ : , 'REVENUE'].round(decimals = 2).sort_values(ascending = False)

We tested the correlation with the continuous features we found. As of now, the highest correlation value her, was AVG_PREP_VID_TIME and TOTAL_PHOTOS_VIEWED

<h2>Data Transformation</h2><br>

<h3>Outlier Startegy</h3><br>


In [4]:
# After exploring data distribution with scatterplots and boxplots, I've noticed outliers
# Outlier detection and strategy

# create a df where values with an absolute Z-score higher than 3 (outliers)
outlier_df = pd.DataFrame(np.abs(stats.zscore(chef_num)) > 3, columns=chef_num.columns)
outlier_columns = outlier_df.sum(axis=0).loc[(outlier_df.sum(axis=0) > 0)].index

#create additional columns with the outliers for each variables, with out_ as prefix
outlier_features = outlier_df[outlier_columns].astype('int').add_prefix('out_')
chef_num = chef_num.join(outlier_features) #join the new columns to our chef_num df


I started by looking at histograms and boxplots of all numerical variables to have a better understanding of their
distribution.
I used the Z-score method to detect and highlight outliers. I created new columns for each variables and their outliers. I then highlighted in my dummy variables which values had an absolute Z-score higer than 3.

<h3>Log Transformation</h3><br>

In [5]:
log_chef_col = ['REVENUE',
                'AVG_TIME_PER_SITE_VISIT',
                'WEEKLY_PLAN',
                'AVG_PREP_VID_TIME',
                'AVG_MEALS_ORDERED',
                'AVG_CLICKS_PER_VISIT',
                'TOTAL_PHOTOS_VIEWED',
                'TOTAL_MEALS_ORDERED',
                'AVG_MEALS_ORDERED',
                'MEDIAN_MEAL_RATING',
                'UNIQUE_MEALS_PURCH',
                'CONTACTS_W_CUSTOMER_SERVICE',
                'PRODUCT_CATEGORIES_VIEWED',
                'CANCELLATIONS_BEFORE_NOON',
                'CANCELLATIONS_AFTER_NOON',
                'PC_LOGINS',
                'MOBILE_LOGINS',
                'EARLY_DELIVERIES',
                'LATE_DELIVERIES',
                'MASTER_CLASSES_ATTENDED',
               ]
# for loop to create new columns with the log transformation
for col in log_chef_col:
    chef_num['log_'+col] = np.log(chef_num[col]+1) #add 1 to avoid doing log of 0
    
# removing original columns now that we have the log columns
chef_num.drop(log_chef_col, axis=1, inplace=True)


I used log transformation method that creates distribution easier to work with and more normally
distributed.

After exploring the distribution visually of our new log variables with the help of scatter plots and box plots,
we noticed that the variable log_TOTAL_PHOTOS_VIEWED still had a clear separation of customer who viewed 
photos and customer who didn't. For that reason, I decided to separate them with a dummy variable.

In [6]:
PHOTOS_VIEWED_zeros = len(chef_num['log_TOTAL_PHOTOS_VIEWED'][chef_num['log_TOTAL_PHOTOS_VIEWED'] == 0])
TOTAL_MEALS_zeros = len(chef_num['log_TOTAL_MEALS_ORDERED'][chef_num['log_TOTAL_MEALS_ORDERED'] == 0])
TIME_SPENT_zeros = len(chef_num['log_AVG_TIME_PER_SITE_VISIT'][chef_num['log_AVG_TIME_PER_SITE_VISIT'] == 0])

#NOT CONTINUOUS MIGHT NEED TO CHANGE TECHNIQUE
UNIQUE_MEALS_zeros = len(chef_num['log_UNIQUE_MEALS_PURCH'][chef_num['log_UNIQUE_MEALS_PURCH'] == 0])

print(f"""
                             No\t\tYes
                           ---------------------
Total Photos Viewed       | {PHOTOS_VIEWED_zeros}\t\t{len(chef) - PHOTOS_VIEWED_zeros}
Total Meals Ordered       | {TOTAL_MEALS_zeros}\t\t{len(chef) - TOTAL_MEALS_zeros}
Avg Time per visit        | {TIME_SPENT_zeros}\t\t{len(chef) - TIME_SPENT_zeros}
Unique meals purchased    | {UNIQUE_MEALS_zeros}\t\t{len(chef) - UNIQUE_MEALS_zeros}
""")


                             No		Yes
                           ---------------------
Total Photos Viewed       | 1140		806
Total Meals Ordered       | 0		1946
Avg Time per visit        | 0		1946
Unique meals purchased    | 0		1946



As we can see from the chart, only Total Photos Viewed has a significant amount of 
values in the "No" column. I will separate both values in a dummy variable.

In [7]:
#dummy variable for viewing photos
chef_num['VIEWED_PHOTOS'] = 0

for index, value in chef_num.iterrows():
    
    if chef_num.loc[index, 'log_TOTAL_PHOTOS_VIEWED'] >0:
        chef_num.loc[index, 'VIEWED_PHOTOS'] = 1

<h3>Categorize Email</h3><br> 

In this step, the goal is to categorize the email domain in different groups. We start by separating the domain from the rest of the email address. We then create lists to separate the domains. We finish by creating dummy variables for each group of emails.

In [8]:
# STEP 1: splitting personal emails

# placeholder list
placeholder_lst = []

# looping over each email address
for index, col in chef_cat.iterrows():
    
    # splitting email domain at '@'
    split_email = chef_cat.loc[index, 'EMAIL'].split(sep = '@')
    
    # appending placeholder_lst with the results
    placeholder_lst.append(split_email)
    

# converting placeholder_lst into a DataFrame 
email_df = pd.DataFrame(placeholder_lst)


# STEP 2: concatenating with original DataFrame

# safety measure in case of multiple concatenations
# remove? chef = pd.read_excel(file)


# renaming column to concatenate
email_df.columns = ['0' , 'personal_email_domain']


# concatenating personal_email_domain with chef DataFrame
chef_cat = pd.concat([chef_cat, email_df['personal_email_domain']],
                   axis = 1)

# set email domain types
professional_email_domains = ['@mmm.com', '@amex.com', '@apple.com',
                                '@boeing.com', '@caterpillar.com', '@chevron.com',
                                '@cisco.com', '@cocacola.com', '@disney.com', 
                                '@dupont.com', '@exxon.com', '@ge.org', '@goldmansacs.com',
                                '@homedepot.com', '@ibm.com', '@intel.com', '@jnj.com', 
                                '@jpmorgan.com','@mcdonalds.com', '@merck.com', '@microsoft.com',
                                '@nike.com', '@pfizer.com', '@pg.com', '@travelers.com',
                                '@unitedtech.com', '@unitedhealth.com', '@verizon.com', 
                                '@visa.com', '@walmart.com']
junk_email_domains = ['@me.com', '@aol.com', '@hotmail.com', '@live.com', '@msn.com', '@passport.com']
personal_email_domains  = ['@gmail.com', '@yahoo.com', '@protonmail.com']




# placeholder list
placeholder_lst = []


# looping to group observations by domain type
for domain in email_df['personal_email_domain']:
        if '@' + domain in professional_email_domains:
            placeholder_lst.append('Professional')
            
        elif '@' + domain in personal_email_domains:
            placeholder_lst.append('Personal')
            
        elif '@' + domain in junk_email_domains:
            placeholder_lst.append('Junk')
            
        else:
            placeholder_lst.append('Unknown')


# concatenating with original DataFrame
chef_cat['domain_group'] = pd.Series(placeholder_lst)


# checking results
# chef_cat['domain_group'].value_counts()

All the emails fall into these 3 categories which make it easier to analyze

In [9]:

########
#creating a bool variable for each domain type
########

#creating a placeholder list that will be used by the following three loops
plc_holder_list_pers = []
plc_holder_list_prof = []
plc_holder_list_junk = []


#creating a variable "Personal_Domain" for personal email
for i in chef_cat["domain_group"]:
    if i == "Personal":
        plc_holder_list_pers.append(1)
    else: 
        plc_holder_list_pers.append(0)
        
chef_cat["Personal_Domain"] = pd.Series(plc_holder_list_pers)

#creating a variable "Professional_Domain" for professional email
for i in chef_cat["domain_group"]:
    if i == "Professional":
        plc_holder_list_prof.append(1)
    else:
        plc_holder_list_prof.append(0)
        
chef_cat["Professional_Domain"] = pd.Series(plc_holder_list_prof)

#creating a variable "Junk_Domain" for Junk email
for i in chef_cat["domain_group"]:
    if i == "Junk":
        plc_holder_list_junk.append(1)
    else:
        plc_holder_list_junk.append(0)
        
chef_cat["Junk"] = pd.Series(plc_holder_list_junk)

#dropping un-needed columns
chef_cat = chef_cat.drop(["NAME", "EMAIL", "FAMILY_NAME", "domain_group", "personal_email_domain"],
             axis = 1)

<h3>Gender Guesser</h3><br> 

After observing that close to 70% of the gender where unknown even after manually modifying
the name that appears multiple times so they get read correctly, we still get a poor results.
Additionlay, the gender.guesser takes a very long time to process so I saved the result in a csv file
but it adds considerable time for python to read it. 
For these reasons, I decided to drop the analysis of this variable.

<h2>Feature Engineering</h2><br>


In [10]:
# cleaning datasets from unnecessary features
chef_model = chef_cat.join(chef_num)

# drop revenue and strings variables
chef_var = chef_model.drop(["log_REVENUE", "FIRST_NAME"], axis = 1)
chef_model = chef_model.drop(["FIRST_NAME"], axis = 1)

<h3>Add additional interesting features</h3><br> 

In [11]:
# Total Photos viewed per login (pc + mobile)
chef_var['PHOTOS_VIEWED_PER_LOGIN'] = chef_var['log_TOTAL_PHOTOS_VIEWED'] / (chef_var['log_MOBILE_LOGINS'] + chef_var['log_PC_LOGINS'])
chef_model['PHOTOS_VIEWED_PER_LOGIN'] = chef_model['log_TOTAL_PHOTOS_VIEWED'] / (chef_model['log_MOBILE_LOGINS'] + chef_model['log_PC_LOGINS'])

# Mater class attended per avg prep vid time
chef_var['NUMN_CLASS_PER_PREP_TIME'] = chef_var['log_MASTER_CLASSES_ATTENDED'] / chef_var['log_AVG_PREP_VID_TIME']
chef_model['NUMN_CLASS_PER_PREP_TIME'] = chef_model['log_MASTER_CLASSES_ATTENDED'] / chef_model['log_AVG_PREP_VID_TIME']

# average number of orders per time/visit
chef_var['AVG_NUMBR_OR_PER_TIME_VISIT'] = chef_var['log_AVG_MEALS_ORDERED'] / chef_var['log_AVG_TIME_PER_SITE_VISIT']
chef_model['AVG_NUMBR_OR_PER_TIME_VISIT'] = chef_model['log_AVG_MEALS_ORDERED'] / chef_model['log_AVG_TIME_PER_SITE_VISIT']



<h3>Standardize the data</h3><br> 

In [12]:
# INSTANTIATING a StandardScaler() object
scaler = StandardScaler()


# FITTING the scaler with the data
scaler.fit(chef_var)


# TRANSFORMING our data after fit
X_scaled = scaler.transform(chef_var)


# converting scaled data into a DataFrame
X_scaled_df = pd.DataFrame(X_scaled)


In [13]:
# adding labels to the scaled DataFrame
X_scaled_df.columns = chef_var.columns

<h3>Combination of Features</h3><br> 

In [14]:
# Create combination of features 

interaction_list = ['log_AVG_TIME_PER_SITE_VISIT',
'log_WEEKLY_PLAN',
'log_CONTACTS_W_CUSTOMER_SERVICE',
'log_AVG_PREP_VID_TIME',
'log_AVG_MEALS_ORDERED',
'log_AVG_CLICKS_PER_VISIT',
'log_TOTAL_PHOTOS_VIEWED',
'log_TOTAL_MEALS_ORDERED',
'log_MEDIAN_MEAL_RATING',
'log_UNIQUE_MEALS_PURCH',
'log_CONTACTS_W_CUSTOMER_SERVICE',
'log_PRODUCT_CATEGORIES_VIEWED',
'log_CANCELLATIONS_BEFORE_NOON',
'log_CANCELLATIONS_AFTER_NOON',
'log_PC_LOGINS',
'log_MOBILE_LOGINS',
'log_EARLY_DELIVERIES',
'log_LATE_DELIVERIES',
'log_MASTER_CLASSES_ATTENDED',
'PHOTOS_VIEWED_PER_LOGIN',
'NUMN_CLASS_PER_PREP_TIME',  
'AVG_NUMBR_OR_PER_TIME_VISIT'
]

interactions = list(combinations(interaction_list, 2))

for interaction in interactions:
    chef_var['{}_{}'.format(interaction_list[0], interaction[1])] = chef_var[interaction[0]]* chef_var[interaction[1]]

We've added quite a few new features, let's see how they impact our model. Adding a variety of interactions between variables is likely to increase the score of the models. I used the method discussed in this [article](https://towardsdatascience.com/feature-engineering-combination-polynomial-features-3caa4c77a755) from Towards Data Science


<h3>Split the data into Train/Split</h3><br> 


In [15]:
# divide chef dataset into dependent and independent variables
X = chef_var
y = chef_model['log_REVENUE']

# Splitting the chef dataset as 75% training and 25% Testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 219)

<h2>Building Predictive Models</h2><br>

<h3>Scikit-Learn Linear Regression</h3><br>

In [16]:
x_variables = ['Personal_Domain', 'Professional_Domain', 'Junk', 'CROSS_SELL_SUCCESS', 'MOBILE_NUMBER', 'TASTES_AND_PREFERENCES', 'PACKAGE_LOCKER', 'REFRIGERATED_LOCKER', 'out_REVENUE', 'out_TOTAL_MEALS_ORDERED', 'out_UNIQUE_MEALS_PURCH', 'out_CONTACTS_W_CUSTOMER_SERVICE', 'out_AVG_TIME_PER_SITE_VISIT', 'out_CANCELLATIONS_BEFORE_NOON', 'out_CANCELLATIONS_AFTER_NOON', 'out_EARLY_DELIVERIES', 'out_LATE_DELIVERIES', 'out_AVG_PREP_VID_TIME', 'out_AVG_MEALS_ORDERED', 'out_MASTER_CLASSES_ATTENDED', 'out_AVG_CLICKS_PER_VISIT', 'out_TOTAL_PHOTOS_VIEWED', 'log_AVG_TIME_PER_SITE_VISIT', 'log_WEEKLY_PLAN', 'log_AVG_PREP_VID_TIME', 'log_AVG_MEALS_ORDERED', 'log_AVG_CLICKS_PER_VISIT', 'log_TOTAL_PHOTOS_VIEWED', 'log_TOTAL_MEALS_ORDERED', 'log_MEDIAN_MEAL_RATING', 'log_UNIQUE_MEALS_PURCH', 'log_CONTACTS_W_CUSTOMER_SERVICE', 'log_PRODUCT_CATEGORIES_VIEWED', 'log_CANCELLATIONS_BEFORE_NOON', 'log_CANCELLATIONS_AFTER_NOON', 'log_PC_LOGINS', 'log_MOBILE_LOGINS', 'log_EARLY_DELIVERIES', 'log_LATE_DELIVERIES',
       'log_MASTER_CLASSES_ATTENDED', 'VIEWED_PHOTOS', 'PHOTOS_VIEWED_PER_LOGIN', 'NUMN_CLASS_PER_PREP_TIME', 'AVG_NUMBR_OR_PER_TIME_VISIT']

# applying scikit-learn

# preparing x-variables from the OLS model
ols_data = chef_model[x_variables]


# preparing response variable
chef_target = chef_model['log_REVENUE']


###############################################
## setting up more than one train-test split ##
###############################################
# FULL X-dataset (normal Y)
x_train_FULL, x_test_FULL, y_train_FULL, y_test_FULL = train_test_split(
            chef_var,     # x-variables
            chef_target,   # y-variable
            test_size = 0.25,
            random_state = 219)


# OLS p-value x-dataset (normal Y)
x_train_OLS, x_test_OLS, y_train_OLS, y_test_OLS = train_test_split(
            ols_data,         # x-variables
            chef_target,   # y-variable
            test_size = 0.25,
            random_state = 219)

In [17]:
# INSTANTIATING a model object
lr = LinearRegression()


# FITTING to the training data
lr_fit = lr.fit(x_train_OLS, y_train_OLS)


# PREDICTING on new data
lr_pred = lr_fit.predict(x_test_OLS)


# SCORING the results
print('OLS Training Score :', lr.score(x_train_OLS, y_train_OLS).round(4))  # using R-square
print('OLS Testing Score  :',  lr.score(x_test_OLS, y_test_OLS).round(4)) # using R-square

lr_train_score = lr.score(x_train_OLS, y_train_OLS).round(4)
lr_test_score  = lr.score(x_test_OLS, y_test_OLS).round(4)

# displaying and saving the gap between training and testing
print('OLS Train-Test Gap :', abs(lr_train_score - lr_test_score).round(4))
lr_test_gap = abs(lr_train_score - lr_test_score).round(4)

OLS Training Score : 0.7473
OLS Testing Score  : 0.7328
OLS Train-Test Gap : 0.0145


In [18]:
# zipping each feature name to its coefficient
lr_model_values = zip(chef_var[x_variables].columns,
                      lr_fit.coef_.round(decimals = 2))


# setting up a placeholder list to store model features
lr_model_lst = [('intercept', lr_fit.intercept_.round(decimals = 2))]


# printing out each feature-coefficient pair one by one
for val in lr_model_values:
    lr_model_lst.append(val)
    

# checking the results
#for pair in lr_model_lst:
#    print(pair)

<h3>Lasso Regression</h3><br>

In [21]:
# INSTANTIATING a model object
lasso_model = sklearn.linear_model.Lasso(alpha = .0001,
                                         normalize = True) # default magitude


# FITTING to the training data
lasso_fit = lasso_model.fit(x_train_FULL, y_train_FULL)

# PREDICTING on new data
lasso_pred = lasso_fit.predict(x_test_FULL)


# SCORING the results
print('Lasso Training Score :', lasso_model.score(x_train_FULL, y_train_FULL).round(4))
print('Lasso Testing Score  :', lasso_model.score(x_test_FULL, y_test_FULL).round(4))


# saving scoring data for future use
lasso_train_score = lasso_model.score(x_train_FULL, y_train_FULL).round(4) # using R-square
lasso_test_score  = lasso_model.score(x_test_FULL, y_test_FULL).round(4)   # using R-square


# displaying and saving the gap between training and testing
print('Lasso Train-Test Gap :', abs(lr_train_score - lr_test_score).round(4))
lasso_test_gap = abs(lr_train_score - lr_test_score).round(4)

Lasso Training Score : 0.7459
Lasso Testing Score  : 0.7323
Lasso Train-Test Gap : 0.0145


In [22]:
# zipping each feature name to its coefficient
lasso_model_values = zip(chef_var.columns, lasso_fit.coef_.round(decimals = 2))


# setting up a placeholder list to store model features
lasso_model_lst = [('intercept', lasso_fit.intercept_.round(decimals = 2))]


# printing out each feature-coefficient pair one by one
for val in lasso_model_values:
    lasso_model_lst.append(val)
    

# checking the results
#for pair in lasso_model_lst:
#    print(pair)

In [23]:
## This code may have to be run more than once ##

# dropping coefficients that are equal to zero

# printing out each feature-coefficient pair one by one
for feature, coefficient in lasso_model_lst:
        
        if coefficient == 0:
            lasso_model_lst.remove((feature, coefficient))

            
# checking the results
#for pair in lasso_model_lst:
#    print(pair)

<h3>ARD Model</h3><br>

In [24]:
# INSTANTIATING a model object
ard_model = sklearn.linear_model.ARDRegression(normalize  = False)


# FITTING the training data
ard_fit = ard_model.fit(x_train_FULL, y_train_FULL)


# PREDICTING on new data
ard_pred = ard_fit.predict(x_test_FULL)


print('Training Score:', ard_model.score(x_train_FULL, y_train_FULL).round(4))
print('Testing Score :',  ard_model.score(x_test_FULL, y_test_FULL).round(4))


# saving scoring data for future use
ard_train_score = ard_model.score(x_train_FULL, y_train_FULL).round(4)
ard_test_score  = ard_model.score(x_test_FULL, y_test_FULL).round(4)


# displaying and saving the gap between training and testing
print('ARD Train-Test Gap :', abs(ard_train_score - ard_test_score).round(4))
ard_test_gap = abs(ard_train_score - ard_test_score).round(4)

Training Score: 0.7595
Testing Score : 0.7517
ARD Train-Test Gap : 0.0078


In [25]:
# zipping each feature name to its coefficient
ard_model_values = zip(chef_var.columns, ard_fit.coef_.round(decimals = 5))


# setting up a placeholder list to store model features
ard_model_lst = [('intercept', ard_fit.intercept_.round(decimals = 2))]


# printing out each feature-coefficient pair one by one
for val in ard_model_values:
    ard_model_lst.append(val)
    

# checking the results
#for pair in ard_model_lst:
#    print(pair)

In [26]:
## This code may have to be run more than once ##

# dropping coefficients that are equal to zero

# printing out each feature-coefficient pair one by one
for feature, coefficient in ard_model_lst:
        
        if coefficient == 0:
            ard_model_lst.remove((feature, coefficient))

            
# checking the results
#for pair in ard_model_lst:
#    print(pair)

In [27]:
# this is the exact code we were using before
X_train_STAND, X_test_STAND, y_train_STAND, y_test_STAND = train_test_split(
            X_scaled_df,
            chef_target,
            test_size = 0.25,
            random_state = 219)

<h3>KNN Optimal number of Neighbors</h3><br>

In [35]:
# creating lists for training set accuracy and test set accuracy
training_accuracy = []
test_accuracy = []


# building a visualization of 1 to 50 neighbors
#neighbors_settings = range(1, 21)


#for n_neighbors in neighbors_settings:
    #Building the model
#    clf = KNeighborsRegressor(n_neighbors = n_neighbors)
#    clf.fit(X_train_STAND, y_train_STAND)
    
    #Recording the training set accuracy
#    training_accuracy.append(clf.score(X_train_STAND, y_train_STAND))
    
    #Recording the generalization accuracy
#    test_accuracy.append(clf.score(X_test_STAND, y_test_STAND))


# plotting the visualization
#fig, ax = plt.subplots(figsize=(12,8))
#plt.plot(neighbors_settings, training_accuracy, label = "training accuracy")
#plt.plot(neighbors_settings, test_accuracy,     label = "test accuracy")
#plt.ylabel("Accuracy")
#plt.xlabel("n_neighbors")
#plt.legend()
#plt.show()


# finding the optimal number of neighbors
#opt_neighbors = test_accuracy.index(max(test_accuracy)) + 1
#print(f"""The optimal number of neighbors is {opt_neighbors}""")

The optimal number of neighbors is 14


I created a visualization from 1 to 50 neighbors to see which value would be optimal. Based on the graph and the output the optimal value is 14.

<h3>KNN Model</h3><br>

In [36]:
# INSTANTIATING a model with the optimal number of neighbors
knn_stand = KNeighborsRegressor(algorithm = 'auto',
                                n_neighbors = 14)



# FITTING the model based on the training data
knn_stand_fit = knn_stand.fit(X_train_STAND, y_train_STAND)



# PREDITCING on new data
knn_stand_pred = knn_stand_fit.predict(X_test_STAND)



# SCORING the results
print('KNN Training Score:', knn_stand.score(X_train_STAND, y_train_STAND).round(4))
print('KNN Testing Score :',  knn_stand.score(X_test_STAND, y_test_STAND).round(4))


# saving scoring data for future use
knn_stand_score_train = knn_stand.score(X_train_STAND, y_train_STAND).round(4)
knn_stand_score_test  = knn_stand.score(X_test_STAND, y_test_STAND).round(4)


# displaying and saving the gap between training and testing
print('KNN Train-Test Gap:', abs(knn_stand_score_train - knn_stand_score_test).round(4))
knn_stand_test_gap = abs(knn_stand_score_train - knn_stand_score_test).round(4)

KNN Training Score: 0.6986
KNN Testing Score : 0.6247
KNN Train-Test Gap: 0.0739


KNN was the model that performed the lowest. It might be interesting to see how well it performs on non-scaled data. We might have a surprise.

In [53]:
model_performance = {
    
    'Model Type'    : ['OLS', 'Lasso', 'ARD', 'KNN'],
           
    'Training' : [lr_train_score, lasso_train_score,
                                   ard_train_score, knn_stand_score_train],
           
    'Testing'  : [lr_test_score, lasso_test_score,
                                   ard_test_score, knn_stand_score_test ],
                    
    'Train-Test Gap' : [lr_test_gap, lasso_test_gap,
                                        ard_test_gap, knn_stand_test_gap],
                    
    'Model Size' : [len(lr_model_lst), len(lasso_model_lst),
                                    len(ard_model_lst), 64],
                    
    'Model' : [lr_model_lst, lasso_model_lst, ard_model_lst, 'NA' ]}


# converting model_performance into a DataFrame
model_performance = pd.DataFrame(model_performance)


In [54]:
print(model_performance)

  Model Type  Training  Testing  Train-Test Gap  Model Size                                              Model
0        OLS    0.7473   0.7328          0.0145          45  [(intercept, 4.93), (Personal_Domain, 0.02), (...
1      Lasso    0.7459   0.7323          0.0145          39  [(intercept, 4.74), (Personal_Domain, 0.01), (...
2        ARD    0.7595   0.7517          0.0078          47  [(intercept, 8.88), (Personal_Domain, 0.01088)...
3        KNN    0.6986   0.6247          0.0739          64                                                 NA


The model whith th highest score is the ARD and it has a very small train-test gap.
The ARD model in this case  explains more than 75% of the variability of the response data around its mean. The ARD model should be used to predict how much revenue Apprentice Chef should expect from each customer within their first year of using the services.
0.75 is a good score for a model and I'm happy with the result.